<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up environment

Clone repository

Install required modules

In [ ]:
!pip install virtualenv

In [ ]:
!virtualenv classifier

In [ ]:
!source /content/classifier/bin/activate; pip list 

Upgrade pip

In [ ]:
!pip install --upgrade pip

Install torch for this machine

In [ ]:
!pip install torch

Install required packages

In [ ]:
!pip install -r requirements.txt

## Working with trained models

### Logistic regression model using tf-df matrix

### Transformer model